In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
#!wget http://www.gutenberg.org/files/100/100-0.txt
!ls
!apt-get install midicsv timidity

# Training

In [0]:
%tensorflow_version 1.x
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np
import os
import random

#Vars
steps = 64
n_hidden = 1024
n_layers = 1
n_input = 129
learning_rate = 0.0003
n_classes = 129
batch_size = 128
iterations = 25000
targetloss = 0.05
currentpath = "/content/drive/My Drive/Programming/rnn/MusicRNN/"
textfile = open(currentpath + "/midis/text.txt", "r")
outfile = open(currentpath + "out.txt", "w")
filecont = list(map(int, textfile.read().replace("\n", "128 ").split()))
flength = len(filecont)

tf.reset_default_graph()
tf.logging.set_verbosity(tf.logging.ERROR) 
    
#FUNCTIONS
#Structure of NN
class RNN:
    def __init__(self, units, goIn, weights, biases):
        self.lstm_layer = rnn.MultiRNNCell([rnn.BasicLSTMCell(units) for i in range(n_layers)])
        self.outputs, self.state = rnn.static_rnn(self.lstm_layer, goIn, dtype = "float32")
        for i in range(n_layers-1):
            self.outputs = [tf.nn.relu(tf.matmul(self.outputs[j], weights["hid"][i]) + biases["hid"][i]) for j in range(len(self.outputs))]
            self.outputs = tf.nn.dropout(self.outputs, keep_prob)
    def p(self, x, keep_prob):
        return tf.matmul(self.outputs[x], weights["out"][0]) + biases["out"][0]

def charnum(c):
    return int(c)

def numtolst(n):
    lst = [0] * n_input
    lst[n] = 1
    return lst

def lisnum(n):
    o = []
    for i in range(len(n)):
        o.append([])
        for j in range(len(n[i])):
            for k in range(len(n[i][j])):
                if n[i][j][k] == 1:
                    o[i].append(k)
    return o

def getBatch():
    outputd = []
    outputl = []
    for i in range(batch_size):
        l = random.randint(0, flength - steps - 1);
        seed = []
        for j in range(steps):
            seed.append(numtolst(charnum(filecont[l + j])))
        outputd.append(list(seed))
        outputl.append(numtolst(charnum(filecont[l + steps])))
    return np.asarray(outputd), np.asarray(outputl)

print("File length is " + str(flength))

weights = {
    "hid" : [tf.Variable(tf.random_normal([n_hidden, n_hidden])), tf.Variable(tf.random_normal([n_hidden, n_hidden])),
             tf.Variable(tf.random_normal([n_hidden, n_hidden])), tf.Variable(tf.random_normal([n_hidden, n_hidden]))],
    "out" : [tf.Variable(tf.random_normal([n_hidden, n_classes]))]
}
biases = {
    "hid" : [tf.Variable(tf.random_normal([n_hidden])), tf.Variable(tf.random_normal([n_hidden])),
             tf.Variable(tf.random_normal([n_hidden])), tf.Variable(tf.random_normal([n_hidden]))],
    "out" : [tf.Variable(tf.random_normal([n_classes]))]
}
keep_prob = tf.placeholder(tf.float32)

inp = tf.placeholder("float", [None, steps, n_input])

lab = tf.placeholder("float", [None, n_classes])

inpu = tf.unstack(inp, steps, 1)

lstm = RNN(n_hidden, inpu, weights, biases)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    logits = lstm.p(-1, keep_prob), labels = lab))

optimize = tf.train.RMSPropOptimizer(learning_rate = learning_rate).minimize(loss)

correct = tf.equal(tf.argmax(lstm.p(-1, keep_prob), 1), tf.argmax(lab, 1))

answer = tf.argmax(lstm.p(-1, keep_prob), 1)

conf = lstm.p(-1, keep_prob)

accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

saver = tf.train.Saver(tf.trainable_variables())

init = tf.global_variables_initializer()

# Run NN
with tf.Session() as sess:
    sess.run(init)
    i = 1
    try:
        print("Loading save")
        saver.restore(sess, currentpath + "weights/weights.ckpt")
    except:
        print("No save found")
    while i <= iterations:
        print("Getting Batches...")
        batch_in, batch_lab = getBatch()
        #print lisnum(batch_in)
        batch_in = batch_in.reshape((batch_size, steps, n_input)) #get input
        fd = {inp: batch_in, lab: batch_lab, keep_prob: 0.3}
        print("Training...")
        sess.run(optimize, fd)
        print("Calculating Accuracy...")
        a = sess.run(accuracy, fd)
        print ("Calculating Loss...")
        l = sess.run(loss, fd)
        #print "Calculating Answers..."
        #ans = sess.run(answer, feed_dict={inp: batch_in, lab: batch_lab})
        #c = sess.run(conf, feed_dict={inp: batch_in, lab: batch_lab})
        print("Iteration " + str(i))
        print("Accuracy  " + str(a)) #display data
        print("Loss      " + str(l))
        #print "Answer    " + str(ans)
        #print "Conf      " + str(c)
        #print "ANS: " + str(sess.run(answer, feed_dict={inp: np.asarray(batch_in), lab: batch_lab}))
        print("\n\n\n")
        if i % 4 == 0:
            spot = saver.save(sess, currentpath + "weights/weights.ckpt")
            print(spot)

        i += 1  
        if l <= targetloss:
            spot = saver.save(sess, currentpath + "weights/weights.ckpt")
            print(spot)
            break;


Calculating Accuracy...
Calculating Loss...
Iteration 2499
Accuracy  0.9140625
Loss      0.35423332




Getting Batches...
Training...
Calculating Accuracy...
Calculating Loss...
Iteration 2500
Accuracy  0.9453125
Loss      0.2228224




/content/drive/My Drive/Programming/rnn/MusicRNN/weights/weights.ckpt
Getting Batches...
Training...
Calculating Accuracy...
Calculating Loss...
Iteration 2501
Accuracy  0.9375
Loss      0.27345905




Getting Batches...
Training...
Calculating Accuracy...
Calculating Loss...
Iteration 2502
Accuracy  0.9609375
Loss      0.2058753




Getting Batches...
Training...
Calculating Accuracy...
Calculating Loss...
Iteration 2503
Accuracy  0.953125
Loss      0.22797267




Getting Batches...
Training...
Calculating Accuracy...
Calculating Loss...
Iteration 2504
Accuracy  0.9453125
Loss      0.24231051




/content/drive/My Drive/Programming/rnn/MusicRNN/weights/weights.ckpt
Getting Batches...
Training...
Calculating Accuracy...
Calculating Loss...
Iteration 25

# Generation

In [0]:
%tensorflow_version 1.x
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np
import os
import sys
from random import randint
#from tensorflow.examples.tutorials.mnist import input_data

#Vars
#mnist=input_data.read_data_sets("/tmp/data/",one_hot=True)
steps = 64
n_hidden = 1024
n_layers = 1
n_input = 129
learning_rate = 0.0001
n_classes = 129
batch_size = 2048
iterations = 4000
beam_width = 2
cbeam = 0
currentpath = "/content/drive/My Drive/Programming/rnn/MusicRNN/"
textfile = open(currentpath + "/midis/text.txt", "r")
filecont = list(map(int, textfile.read().replace("\n", "128 ").split()))
outfile = open(currentpath + "output.txt", "w")
let = steps
seed = []
rawseed = []
paths = [[0]] * beam_width
pathsref = [[0]] * beam_width

tf.reset_default_graph()
tf.logging.set_verbosity(tf.logging.ERROR)
        
#FUNCTIONS
#Structure of NN
class RNN:
    def __init__(self, units, goIn, weights, biases):
        self.lstm_layer = rnn.MultiRNNCell([rnn.BasicLSTMCell(units) for i in range(n_layers)])
        self.outputs, self.state = rnn.static_rnn(self.lstm_layer, goIn, dtype = "float32")
        for i in range(n_layers-1):
            self.outputs = [tf.nn.relu(tf.matmul(self.outputs[j], weights["hid"][i]) + biases["hid"][i]) for j in range(len(self.outputs))]
            self.outputs = tf.nn.dropout(self.outputs, keep_prob)
    def p(self, x, keep_prob):
        return tf.matmul(self.outputs[x], weights["out"][0]) + biases["out"][0]

def charnum(c):
    if c == "\n":
        return 128
    else:
        return int(c)

def numchar(c):
    if c == 128:
        return "\n"
    else:
        return str(c) + " "

def numtolst(n):
    lst = [0] * n_input
    lst[n] = 1
    return lst

def skewedrandom():
    ret = randint(0, beam_width-1)
    return ret

def getBatch():
    return np.asarray(seed)


seedstart = randint(10000, 50000)

for i in range(steps):
    seed.append(numtolst(charnum(filecont[i + seedstart])))
    rawseed.append(filecont[i + seedstart])

weights = {
    "hid" : [tf.Variable(tf.random_normal([n_hidden, n_hidden])), tf.Variable(tf.random_normal([n_hidden, n_hidden])),
             tf.Variable(tf.random_normal([n_hidden, n_hidden])), tf.Variable(tf.random_normal([n_hidden, n_hidden]))],
    "out" : [tf.Variable(tf.random_normal([n_hidden, n_classes]))]
}
biases = {
    "hid" : [tf.Variable(tf.random_normal([n_hidden])), tf.Variable(tf.random_normal([n_hidden])),
             tf.Variable(tf.random_normal([n_hidden])), tf.Variable(tf.random_normal([n_hidden]))],
    "out" : [tf.Variable(tf.random_normal([n_classes]))]
}

keep_prob = tf.placeholder(tf.float32)

inp = tf.placeholder("float", [None, steps, n_input])

inpu = tf.unstack(inp, steps, 1)

lstm = RNN(n_hidden, inpu, weights, biases)

ans_sing = lstm.p(-1, keep_prob)

answers = tf.nn.top_k(lstm.p(-1, keep_prob), n_classes)

ans_conf = answers[0]
ans_val = answers[1]

saver = tf.train.Saver(tf.trainable_variables())

init = tf.global_variables_initializer()

# Run NN
with tf.Session() as sess:
    sess.run(init)
    i = 1
    try:
        print("Loading save")
        saver.restore(sess, currentpath + "weights/weights.ckpt")
    except:
        print("No save found")
    for j in range(len(rawseed)):
        sys.stdout.write(numchar(rawseed[j]))
    sys.stdout.write("||")
    while i <= iterations:
        if i % 200 == 0:
            print("Epoch " + str(i) + " / " + str(iterations))
        batch_in = getBatch()
        batch_in = batch_in.reshape((1, steps, n_input))
        pred = sess.run(ans_val, {inp: batch_in, keep_prob: 1})
        randomizedpred = pred[0][int(np.absolute(np.random.normal(scale=0.7)))]
        seed.pop(0)
        seed.append(numtolst(randomizedpred))
        #print(pred[0])
        sys.stdout.write(numchar(randomizedpred))
        outfile.write(numchar(randomizedpred))
        i += 1
        #Failed Beam Search Implementation below USE AT OWN RISK (ITS KINDA BAD)
        '''updc = []
        updv = []
        kp = 0.9
        if cbeam == 0:
            batch_in = batch_in.reshape((1, steps, n_input))
            fd = {inp: batch_in, keep_prob: kp}
            ansv = sess.run(ans_val, fd)[0]
            ansc = sess.run(ans_conf, fd)[0]
            for j in range(beam_width):
                paths[j] = [ansc[j], ansv[j]]
        for j in range(beam_width):
            batch_in = getBatch()
            if cbeam == 0:
                break;
            for k in range(cbeam):
                batch_in = batch_in[1:]
            for k in range(cbeam):
                batch_in = batch_in.tolist()
                batch_in.append(numtolst(pathsref[j][k + 1]))
                batch_in = np.array(batch_in)
            #print lisnum(batch_in)
            batch_in = batch_in.reshape((1, steps, n_input)) #get input
            fd = {inp: batch_in, keep_prob: kp}
            ansv = sess.run(ans_val, fd)[0]
            ansc = sess.run(ans_conf, fd)[0]
            for k in range(beam_width):
                updc.append(pathsref[j][0] + ansc[k])
                updv.append(ansv[k])
        
        topcs = []
        topvs = []
        
        for j in range(beam_width):
            if cbeam == 0:
                break;
            topc = 0
            topv = 0
            topi = 0
            for k in range(beam_width):
                if updc[k] > topc:
                    topc = updc[k]
                    topv = updv[k]
                    topi = k
            topcs.append(topc)
            topvs.append(topv)
            temp = [topc]
            for k in range(cbeam):
                temp.append(pathsref[j][k+1])
            temp.append(topv)
            paths[j] = temp
            updc.pop(topi)
            updv.pop(topi)
        
        i += 1 
        cbeam += 1
        pathsref = paths
        if cbeam >= beam_width:
            ind = skewedrandom()
            print("IND: " + str(ind))
            for j in range(beam_width):
                printer = ""
                if j == 0:
                    printer = paths[ind][j + 1]
                else:
                    printer = paths[ind][j + 1]
                seed.pop(0)
                seed.append(numtolst(paths[ind][j + 1]))
                sys.stdout.write(numchar(paths[ind][j + 1]))
                outfile.write(numchar(paths[ind][j + 1]))
            paths = [[0]] * beam_width
            pathsref = [[0]] * beam_width
            cbeam = 0'''
outfile.close()


# Clean Up Output File

In [0]:
import os

rfile = "/content/drive/My Drive/Programming/rnn/MusicRNN/output"
rfilebk = "/content/drive/My\ Drive/Programming/rnn/MusicRNN/output"

infile = open(rfile + ".txt", "r")
outfile = open(rfile + ".tmp.txt", "w+")

for note in infile:
    line = note.split(" ")
    line = line[:-1]
    already = []
    if len(line) == 0:
        outfile.write("\n")
        continue
    for i in line:
        if i in already:
            continue
        if not i.isdigit():
            continue
        already.append(i)
    already.sort()
    for i in already:
        outfile.write(i + " ")
    outfile.write("\n")
outfile.close()

# Convert From Text To Midi

In [0]:
import os
from time import sleep

rfile = "/content/drive/My Drive/Programming/rnn/MusicRNN/output"
rfilebk = "/content/drive/My\ Drive/Programming/rnn/MusicRNN/output"

ninterval = 140

infile = open(rfile + ".tmp.txt", "r")
outfile = open(rfile + ".tmp", "w+")
outfile.write("0, 0, Header, 0, 1, 384\n")
outfile.write("1, 0, Start_track\n")
outfile.write('1, 0, Title_t, "Starttrack"\n')
outfile.write("1, 0, Time_signature, 4, 2, 24, 8\n")
outfile.write("1, 0, Tempo, 600000\n")
outfile.write("1, 0, Program_c, 1, 0\n")
outfile.write('1, 0, Instrument_name_t, "ahsdfhalisudhgaliughlu"\n')
time = -ninterval
prev = []
ons = 0
ofs = 0
for note in infile:
    time += ninterval
    line = note.split(" ")
    line = line[:-1]
    already = []
    if len(line) == 0:
        continue
    for i in prev:
        if not i.isdigit():
            continue
        isDel = True
        for j in line:
            if not j.isdigit():
                continue
            if i == j:
                isDel = False
                break
        if isDel:
            outfile.write("1, " + str(time) + ", Note_off_c, 1, " + str(i) + ", 0\n")
            ofs += 1
    for i in line:
        if i in already:
            continue
        if not i.isdigit():
            continue
        isNew = True
        already.append(i)
        for j in prev:
            if not j.isdigit():
                continue
            if i == j:
                isNew = False
                break
        if isNew:
            outfile.write("1, " + str(time) + ", Note_on_c, 1, " + str(i) + ", 127\n")
            ons += 1
    prev = already
for i in prev:
    if not i.isdigit():
        continue
    outfile.write("1, " + str(time) + ", Note_off_c, 1, " + str(i) + ", 0\n")
    ofs += 1
print(ons)
print(ofs)
outfile.write("1, " + str(time) + ", End_track\n")
outfile.write("0, 0, End_of_file\n")
outfile.close()
print(os.popen("csvmidi " + rfilebk + ".tmp > " + rfilebk + "out.mid").read())
sleep(1)
print(os.popen("timidity -Ow '" + rfile + "out.mid'").read())

# Convert From Midi To Text

In [0]:
!ls /content/drive/My\ Drive/Programming/rnn/MusicRNN/midis/*.mid -1 | wc -l
#Convert midi to txt
import os

rootdirnobk = "/content/drive/My Drive/Programming/rnn/MusicRNN/midis/"
rootdiryebk = "/content/drive/My\ Drive/Programming/rnn/MusicRNN/midis/"

files = int(os.popen("ls " + rootdiryebk + "*.mid -1 | wc -l").read())

print(files)

songs = []

for i in range(files):
    print("READING " + str(i))
    try:
        songs.append(os.popen("midicsv " + rootdiryebk + str(i) + ".mid").read())
    except:
        print("BAD" + str(i))

shuffle(songs)

print(songs[0])

outfile = open(rootdirnobk + "text.txt", "w+")

for i in range(len(songs)):
    #outfile.write("[SONG # " + str(i) + "]")
    notes = [0] * 128
    out = ""
    
    csong = songs[i].split("\n")
    final = 0
    stop = len(csong)
    ret = ""
    ptime = 0
    div = 1
    for j in range(stop):
        words = csong[j].split(", ");
        if len(words) < 3:
            continue
        if words[2] == "Tempo":
            div = int(words[3]) / 10000
    for j in range(stop):
        words = csong[j].split(", ");
        if len(words) < 3:
            continue
        if words[2] == "Note_off_c" or words[2] == "Note_on_c":
            final = max(final, int(words[1]))
    time = [""] * int((final+1) / div)
    for j in range(stop):
        n = 0
        words = csong[j].split(", ");
        if len(words) < 3:
            continue
        
        if words[2] == "Note_off_c" or words[2] == "Note_on_c":
            for k in range(len(notes)):
                if notes[k] == 1:
                    for l in range(int(ptime / div), int(int(words[1]) / div)):
                        time[l] += str(k) + " "
            ptime = int(words[1])
        if words[2] == "Note_on_c":
            notes[int(words[4])] = 1
        if words[2] == "Note_off_c":
            notes[int(words[4])] = 0
    toolong = False
    for j in range(len(time)):
        outline = time[j].split(" ")
        if(len(outline) > 10):
            toolong = True
            print("OMIT: " + str(i))
            break
    if toolong:
        continue
    for j in range(len(time)):
        outfile.write(time[j] + "\n")
outfile.close()
print("done")
